In [3]:
!mkdir '../datasets/sentence_level_lstm'

In [6]:
!ls '../datasets'

LSTM  LSTM_1  sentence_level_lstm


In [210]:
from io import StringIO
import requests
import pandas as pd
import numpy as np
import re, random, collections, json
import tensorflow as tf
from tensorflow.contrib import rnn

In [211]:
# The code was removed by DSX for sharing.

0     After waiting what seemed like  G52   forever...
1     Yeah   G15   it was such a change! I  G40   s...
2       G13  And I was so glad that all of the meas...
3     Ben and I had been saying for the past two we...
4     Of course   G20  that is what happened for th...
Name: sentence, dtype: object

In [212]:
#get every sentence and put it into an array of separated words
sentences_list = df_data.values.tolist()
sentences_separated = [ re.findall(r"[\w']+|[.,!?;]", sent[0].lower()) for sent in sentences]
sentences_separated = np.array(sentences_separated)

avrgs = [len(x) for x in sentences_separated]
max_sentence_length = np.max(avrgs)
print('longest sentence: ' + str(max_sentence_length))

longest sentence: 160


In [213]:
def filter_gestures(word):
    ges = re.search(r'g\d{1,2}',word)
    if ges is None:
        return True
    else:
        return False

def build_dictionary(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)+1
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

'''make the list of list of words into list of words'''
words = [item for sublist in sentences_separated for item in sublist]

'''filter the gesture annotations'''
words = [x for x in words if filter_gestures(x)]

'''count the unique appearance of words and assign them a number'''
dictionary, reverse_dictionary = build_dictionary(words)


#print(dictionary)
vocab_size = len(dictionary)
print(vocab_size)

3214


In [225]:
'''fixed variables'''
#sequence length
time_steps=max_sentence_length

#self explanatory
input_size=1

#57 getsure types + no gesture
n_classes=58
no_gesture_index = n_classes - 1

'''replaceable variables'''
#could be any number
num_units=30

#we need to learn about this
learning_rate=0.001

#tbd
batch_size=20

#number of iterations
epoch = 800/batch_size

In [226]:
def populate_output(gesture_list):
    
    single_output = [0]*n_classes
    
    #if this is a no getsure sentence mark the last cell
    if len(gesture_list) == 0:
        single_output[no_gesture_index] = 1
    
    #else create their possiilities (try with giving them all 1 and then making them sum up to 1)
    else:
        #for a correct probability distribution
        #count_gestures = len(gesture_list)
        #value = 1/count_gestures
        #for ges in gesture_list:
        #    single_output[ges] = value
        
        #for a multiclass classification
        for ges in gesture_list:
            single_output[ges] = 1
    
    return single_output

def create_input_output(sentence):
    
    #create the max length of an empty array filled with 0s
    single_input = [0]*max_sentence_length
    loc = 0
    
    #create an array to keep the gesture information
    gesture_list = []
    
    #for every word in the sentence
    for word in sentence:
        
        #if word is not a gesture
        gesture = re.search(r'g\d{1,2}', word)
        
        if gesture is None:
            
            #find the encoded version of the word and add it to the corresponding space
            encoded_word = dictionary[word]
            single_input[loc] = encoded_word
            loc = loc + 1
            
        #if the word is a gesture
        else:

            #add it to the gesture list
            num_gesture = int(word[1:])
            gesture_list.append(num_gesture)

    #when the sentences traversing is done populat ethe output array for this specific input
    single_output = populate_output(gesture_list)

    return single_input, single_output


In [227]:
model_data_input = []
model_data_output = []

for sent in sentences_separated:
    in_data, out_data = create_input_output(sent)
    model_data_input.append(in_data)
    model_data_output.append(out_data)
    
len(model_data_input)

874

In [228]:
#Random sampling training and test data
_TRAINING_INS_COUNT = 800
train_indexes = random.sample(range(0, len(model_data_input)), _TRAINING_INS_COUNT)
test_indexes = [i for i in range(0, len(model_data_input)) if not i in train_indexes]


#creating training data 80%
_TRAINING_INPUT = np.array(model_data_input)
_TRAINING_INPUT = _TRAINING_INPUT[train_indexes]
_TRAINING_OUTPUT = np.array(model_data_output)
_TRAINING_OUTPUT = _TRAINING_OUTPUT[train_indexes]


#creating testing data 20%
_TESTING_INPUT = np.array(model_data_input)
_TESTING_INPUT = _TESTING_INPUT[test_indexes]
_TESTING_OUTPUT = np.array(model_data_output)
_TESTING_OUTPUT = _TESTING_OUTPUT[test_indexes]


In [229]:

tf.reset_default_graph()

out_weights=tf.Variable(tf.random_normal([num_units,n_classes]), name="weights")
out_bias=tf.Variable(tf.random_normal([n_classes]), name="biases")

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,input_size])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,input_size] to "time_steps" number of [batch_size,input_size] tensors
input=tf.unstack(x ,time_steps,1)

In [230]:
# defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)

with tf.variable_scope("rnn", reuse=None):
    outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")
    print(outputs[-1])
    
    #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
    prediction=tf.matmul(outputs[-1],out_weights)+out_bias

    #loss_function
    loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction,labels=y))
    #optimization
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    #model evaluationmatmul
    correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_479:0", shape=(?, 30), dtype=float32)


In [231]:
#alternative model evaluation
def alternative_accuracy(raw_predictions, labels):
    
    accuracy_list = []
    
    for i in range(0,batch_size):
        
        #copy the prediction list for problem free manipulation
        temp_predictions = raw_predictions[i]
        
        #if this is not a no gesture sentence
        if labels[i][no_gesture_index] != 1:
            
            #get the number of gestures in this sentence
            count_of_gestures = sum(labels[i])
            
            #array to keep the indices of max n predictions
            predicted_indices = []
            true_indices = []

            #loop to get the highest n predicted indices
            for j in range(0, count_of_gestures):
                max_index = temp_predictions.index(max(temp_predictions))
                predicted_indices.append(max_index)
                temp_predictions[max_index] = -1000
                
            #loop to get the indices of actual values
            for k in range(0,n_classes):
                if labels[i][k] == 1:
                    true_indices.append(k)
            
            #list of indices with the highes tpredictions = predicted_indices
            #list of actual values = true_indices
            
            for prediction in predicted_indices:
                if prediction in true_indices:
                    accuracy_list.append(1)
                else:
                    accuracy_list.append(0)
                    
        #if this is a no getsure sentence:
        else:
            if temp_predictions[no_gesture_index] == 1:
                accuracy_list.append(1)
            else:
                accuracy_list.append(0)
            
    batch_accuracy = np.mean(accuracy_list)
    return batch_accuracy         
    

In [233]:
#initialize variables
batch_init = 0
limit = batch_init + batch_size
init=tf.global_variables_initializer()
#saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(init)
    iter=0
    while iter<epoch:
        
        batch_x = _TRAINING_INPUT[batch_init: limit]
        batch_y = _TRAINING_OUTPUT[batch_init: limit]
        batch_init = batch_init + batch_size
        limit = batch_init + batch_size

        batch_x=batch_x.reshape((batch_size,time_steps,input_size))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})
        #print(sess.run(esselamualeykum,feed_dict={x:batch_x,y:batch_y}))
        
        raw_predictions = prediction.eval(feed_dict={x: batch_x}, session=sess).tolist()
        #results = [sublist.index(max(sublist)) for sublist in raw_predictions]
        #print(results)
        alt_acc = alternative_accuracy(raw_predictions, batch_y)
        print("Alternative accuracy ", str(alt_acc))

        if iter %1==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1        
    
    #save_path = saver.save(sess,"../datasets/LSTM_1/model.ckpt")
    #print(save_path)

    #calculating test accuracy
    test_data = _TESTING_INPUT.reshape((-1, time_steps, input_size))
    test_label = _TESTING_OUTPUT
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
    raw_predictions_test = prediction.eval(feed_dict={x: test_data}, session=sess).tolist()
    alt_acc_test = alternative_accuracy(raw_predictions, batch_y)
    print("Alternative accuracy for the test ", str(alt_acc_test))
        
    #print("Prediction:", sess.run(prediction, feed_dict={x: test_data, y: test_label}))
    #d = prediction.eval(feed_dict={x: test_data}, session=sess)
    #print ("predictions", d)
    #print(type(d[0]))

Alternative accuracy  0.0175438596491
For iter  0
Accuracy  0.0
Loss  0.743864
__________________
Alternative accuracy  0.06
For iter  1
Accuracy  0.0
Loss  0.731054
__________________
Alternative accuracy  0.115384615385
For iter  2
Accuracy  0.0
Loss  0.731542
__________________
Alternative accuracy  0.0535714285714
For iter  3
Accuracy  0.05
Loss  0.732551
__________________
Alternative accuracy  0.0819672131148
For iter  4
Accuracy  0.0
Loss  0.742845
__________________
Alternative accuracy  0.0625
For iter  5
Accuracy  0.0
Loss  0.728739
__________________
Alternative accuracy  0.0757575757576
For iter  6
Accuracy  0.0
Loss  0.738041
__________________
Alternative accuracy  0.0425531914894
For iter  7
Accuracy  0.0
Loss  0.716666
__________________
Alternative accuracy  0.0833333333333
For iter  8
Accuracy  0.0
Loss  0.706484
__________________
Alternative accuracy  0.037037037037
For iter  9
Accuracy  0.0
Loss  0.709531
__________________
Alternative accuracy  0.0701754385965
For